# How to stream chat model responses


All [chat models](https://python.langchain.com/v0.2/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html) implement the [Runnable interface](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable), which comes with a **default** implementations of standard runnable methods (i.e. `ainvoke`, `batch`, `abatch`, `stream`, `astream`, `astream_events`).

The **default** streaming implementation provides an`Iterator` (or `AsyncIterator` for asynchronous streaming) that yields a single value: the final output from the underlying chat model provider.

:::{.callout-tip}

The **default** implementation does **not** provide support for token-by-token streaming, but it ensures that the the model can be swapped in for any other model as it supports the same standard interface.

:::

The ability to stream the output token-by-token depends on whether the provider has implemented proper streaming support.

See which [integrations support token-by-token streaming here](/docs/integrations/chat/).

## Sync streaming

Below we use a `|` to help visualize the delimiter between tokens.

In [1]:
from langchain_anthropic.chat_models import ChatAnthropic

chat = ChatAnthropic(model="claude-3-haiku-20240307")
for chunk in chat.stream("Write me a 1 verse song about goldfish on the moon"):
    print(chunk.content, end="|", flush=True)

|

Here's|

 a one|

-|

verse song about gol|

...


Exploring|

 cr|

aters, a sight|

 to behold.|

|

## Async Streaming

In [2]:
from langchain_anthropic.chat_models import ChatAnthropic

chat = ChatAnthropic(model="claude-3-haiku-20240307")
async for chunk in chat.astream("Write me a 1 verse song about goldfish on the moon"):
    print(chunk.content, end="|", flush=True)

|

Here is a |

1 verse song|

 about goldfish on|

 the moon:|

...

 free,|

 
Swimming|

 through et|

ernity.|

|

## Astream events

Chat models also support the standard [astream events](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.astream_events) method.

This method is useful if you're streaming output from a larger LLM application that contains multiple steps (e.g., an LLM chain composed of a prompt, llm and parser).

In [3]:
from langchain_anthropic.chat_models import ChatAnthropic

chat = ChatAnthropic(model="claude-3-haiku-20240307")
idx = 0

async for event in chat.astream_events(
    "Write me a 1 verse song about goldfish on the moon", version="v1"
):
    idx += 1
    if idx >= 5:  # Truncate the output
        print("...Truncated")
        break
    print(event)

{'event': 'on_chat_model_start', 'run_id': 'a1ad1e49-79f7-4c7b-85af-62b447076e8b', 'name': 'ChatAnthropic', 'tags': [], 'metadata': {}, 'data': {'input': 'Write me a 1 verse song about goldfish on the moon'}, 'parent_ids': []}


{'event': 'on_chat_model_stream', 'run_id': 'a1ad1e49-79f7-4c7b-85af-62b447076e8b', 'tags': [], 'metadata': {}, 'name': 'ChatAnthropic', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-a1ad1e49-79f7-4c7b-85af-62b447076e8b', usage_metadata={'input_tokens': 21, 'output_tokens': 0, 'total_tokens': 21})}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': 'a1ad1e49-79f7-4c7b-85af-62b447076e8b', 'tags': [], 'metadata': {}, 'name': 'ChatAnthropic', 'data': {'chunk': AIMessageChunk(content='Here is a ', additional_kwargs={}, response_metadata={}, id='run-a1ad1e49-79f7-4c7b-85af-62b447076e8b')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': 'a1ad1e49-79f7-4c7b-85af-62b447076e8b', 'tags': [], 'metadata': {}, 'name': 'ChatAnthropic', 'data': {'chunk': AIMessageChunk(content='1 verse song', additional_kwargs={}, response_metadata={}, id='run-a1ad1e49-79f7-4c7b-85af-62b447076e8b')}, 'parent_ids': []}
...Truncated
